In [2]:
# 로컬 환경에 설치가 안되어있다면 실행
pip install transformers datasets scikit-learn torch


   ---------------------------------------- 0.0/25.2 MB ? eta -:--:--
   --- ------------------------------------ 2.1/25.2 MB 9.8 MB/s eta 0:00:03
   ------ --------------------------------- 4.2/25.2 MB 9.7 MB/s eta 0:00:03
   --------- ------------------------------ 6.3/25.2 MB 9.9 MB/s eta 0:00:02
   ------------- -------------------------- 8.4/25.2 MB 9.8 MB/s eta 0:00:02
   ---------------- ----------------------- 10.2/25.2 MB 9.5 MB/s eta 0:00:02
   ------------------- -------------------- 12.1/25.2 MB 9.4 MB/s eta 0:00:02
   ---------------------- ----------------- 13.9/25.2 MB 9.2 MB/s eta 0:00:02
   ------------------------ --------------- 15.7/25.2 MB 9.2 MB/s eta 0:00:02
   --------------------------- ------------ 17.6/25.2 MB 9.1 MB/s eta 0:00:01
   ------------------------------ --------- 19.4/25.2 MB 9.0 MB/s eta 0:00:01
   --------------------------------- ------ 21.2/25.2 MB 9.1 MB/s eta 0:00:01
   ------------------------------------ --- 23.1/25.2 MB 9.0 MB/s eta 0:00:

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.7.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.12.2 which is incompatible.
weasel 0.3.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.


# 아래는 모델을 불러오는 코드
### path에 모델을 불러오고, 경로는 이 코드와 같은 경로에 두면 된다.
### sentences 변수를 통해 직접 문장을 다수 테스트해볼 수 있다.

In [44]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModelForSequenceClassification
import os

# ✅ 모델 경로
path = "finetuned_data(3)"
slot_model_path = f"./{path}/slot"
intent_model_path = f"./{path}/intent"
tokenizer_path = f"./{path}/tokenizer"

# ✅ 레이블 리스트
label_list = ['B-DIRECTION', 'B-LINE', 'B-ROUTE', 'B-STATION', 'B-TRANSPORT-BUS', 'B-TRANSPORT-SUBWAY', 'O']
intent_list = ['arrival_bus', 'arrival_subway', 'congestion', 'other']

# ✅ 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ 토크나이저 및 모델 불러오기
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
slot_model = AutoModelForTokenClassification.from_pretrained(slot_model_path).to(device)
intent_model = AutoModelForSequenceClassification.from_pretrained(intent_model_path).to(device)

# 모델 1: 슬롯 태깅용 (Token Classification)
slot_model = AutoModelForTokenClassification.from_pretrained(slot_model_path).to(device)
slot_model.eval()

# 모델 2: 인텐트 분류용 (Sequence Classification)
intent_model = AutoModelForSequenceClassification.from_pretrained(intent_model_path).to(device)
intent_model.eval()

import torch
import torch.nn.functional as F
from transformers import AutoTokenizer

def predict(sentence, tokenizer, slot_model, intent_model, label_list, intent_list):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 문장을 단어 단위로 분할 (띄어쓰기 기준)
    words = sentence.strip().split()

    # 토큰화 (단어 단위 입력!)
    tokenized = tokenizer(
        words,
        is_split_into_words=True,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128
    )

    # ⚠️ tokenized를 dict로 바꾸지 말고 tensor만 device로 이동
    for k in tokenized:
        tokenized[k] = tokenized[k].to(device)

    # 모델을 device로
    slot_model.to(device)
    intent_model.to(device)

    with torch.no_grad():
        # 🔸 인텐트 예측
        intent_logits = intent_model(**tokenized).logits
        intent_probs = F.softmax(intent_logits, dim=1)[0]
        intent_pred_id = torch.argmax(intent_probs).item()
        intent_label = intent_list[intent_pred_id]
        intent_score = intent_probs[intent_pred_id].item()

        # 🔸 슬롯 태깅 예측
        slot_logits = slot_model(**tokenized).logits  # (1, seq_len, num_labels)
        slot_probs = F.softmax(slot_logits, dim=2)[0]
        slot_preds = torch.argmax(slot_probs, dim=1).tolist()
        slot_scores = slot_probs[range(len(slot_preds)), slot_preds].tolist()

    input_ids = tokenized["input_ids"][0].cpu()
    word_ids = tokenized.word_ids(batch_index=0)
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # 단어 단위 병합 및 출력
    print(f"\n🟦 문장: {sentence}")
    print(f"🔸 예측 인텐트: {intent_label}  (score: {intent_score:.4f})")
    print(f"🔸 슬롯 태깅:")

    word_to_tag = {}
    for idx, word_id in enumerate(word_ids):
        if word_id is None or input_ids[idx].item() in tokenizer.all_special_ids:
            continue
        if word_id not in word_to_tag:
            pred_id = slot_preds[idx]
            score = slot_scores[idx]
            word_to_tag[word_id] = (label_list[pred_id], score)

    for i, word in enumerate(words):
        if i in word_to_tag:
            tag, score = word_to_tag[i]
            print(f"   {word:10} → {tag:20} (score: {score:.4f})")
        else:
            print(f"   {word:10} → [NO TAG]")

    print()


In [45]:

# ✅ 테스트 문장
sentences = [
    "단국대입구 정류소에 720번 버스 지금 오고 있어?",
    "죽전역에서 신분당선 타면 강남 방면으로 얼마나 걸려?",
    "지금 9호선 급행열차 타면 여의도 쪽 사람 많아?",
    "110-2번 버스가 동인천 방향으로 곧 도착하나요?",
    "양재역에서 지하철 타고 삼성 방면 가는 거 얼마나 복잡할까?",
    "지하철역 플랫폼에 사람 너무 많아서 숨막힐 뻔했어.",
    "파란불 안 켜져 있으면 버스 안 오는 거야?",
    "강남역에서 타는 2호선 지금 헬게이트일까?",
    "7017번은 신설동역 근처 언제쯤 지나가나요?",
    "양재시민의숲역에 오늘 무슨 행사 있대요?",
    "3호선 구파발 방면 타면 사람 많을까요?",
    "양재 시민의 숲에서 420번 버스를 타면 강남역 근처 맛집에 갈 수 있다더라.",
    "서울대입구역 정류소에 6511번 버스 곧 도착하나요?",
    "지금 시청역에서 탈 수 있는 2호선 열차는 어디까지 도착했나요?",
    "양재 시민의숲 역에서 510 버스는 대체 언제 오는거야? 얼마나 기다려야 도착하는지 알아봐줄래?"
]
sentences = [
    "혜화역 근처에서 273번 버스 지금쯤 도착했을까?",
    "출근 시간에 9호선 타면 고속터미널역 쪽 사람 엄청 많지 않아?",
    "버스 정류장에 파란불 안 들어와 있으면 7011번 안 오는 거야?",
    "목동역에서 타는 5호선 어디까지 도착했는지 확인해줄래?",
    "강남에서 사당 방향 가는 2호선 지금 얼마나 붐비는지 궁금하네.",
    "연세대앞 정류소에 163번 버스 몇 분 안에 도착하나요?",
    "지하철 에스컬레이터가 고장 났다는데 다른 출입구로 갈 수 있어?",
    "삼성역으로 가려면 어떤 노선 타야 빨라?"
]
sentences = [
    "강남역에서 내려서 삼성역 방면으로 가는 2호선은 언제 와?",
    "잠실에서 3318버스를 타고 문정역 근처에 가려면 얼마나 걸려?",
    "시청역 앞에서 1호선 급행열차 타면 인천 방향으로 금방 가?",
    "수서역까지 가려면 신분당선이랑 3호선 중에 뭐 타는 게 빠를까?",
    "버스정류장에서 파란불 안 켜지면 421번 버스 못 오는 거야?"
]



# ✅ 예측 실행
for sentence in sentences:
    predict(sentence, tokenizer, slot_model, intent_model, label_list, intent_list)


🟦 문장: 강남역에서 내려서 삼성역 방면으로 가는 2호선은 언제 와?
🔸 예측 인텐트: arrival_subway  (score: 1.0000)
🔸 슬롯 태깅:
   강남역에서      → B-STATION            (score: 1.0000)
   내려서        → O                    (score: 1.0000)
   삼성역        → B-DIRECTION          (score: 0.9999)
   방면으로       → O                    (score: 1.0000)
   가는         → O                    (score: 1.0000)
   2호선은       → B-LINE               (score: 1.0000)
   언제         → O                    (score: 1.0000)
   와?         → O                    (score: 1.0000)


🟦 문장: 잠실에서 3318버스를 타고 문정역 근처에 가려면 얼마나 걸려?
🔸 예측 인텐트: arrival_bus  (score: 1.0000)
🔸 슬롯 태깅:
   잠실에서       → B-STATION            (score: 1.0000)
   3318버스를    → B-ROUTE              (score: 1.0000)
   타고         → O                    (score: 1.0000)
   문정역        → B-STATION            (score: 1.0000)
   근처에        → O                    (score: 1.0000)
   가려면        → O                    (score: 1.0000)
   얼마나        → O                    (score: 1.0000)
   걸려?        → O     

# 아래 역시 모델을 불러와 테스트해보는 코드
### path에 모델을 불러오고, 경로는 이 코드와 같은 경로에 두면 된다.
### 단, 직접 입력하여 실시간으로 문장을 입력하고 테스트해볼 수 있다.

In [38]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModelForSequenceClassification

# ✅ 모델 경로
path = "finetuned_data(4)"
slot_model_path = f"./{path}/slot"
intent_model_path = f"./{path}/intent"
tokenizer_path = f"./{path}/tokenizer"

# ✅ 레이블 리스트
label_list = ['B-DIRECTION', 'B-LINE', 'B-ROUTE', 'B-STATION', 'B-TRANSPORT-BUS', 'B-TRANSPORT-SUBWAY', 'O']
intent_list = ['arrival_bus', 'arrival_subway', 'congestion', 'other']

# ✅ 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ 토크나이저 및 모델 불러오기
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
slot_model = AutoModelForTokenClassification.from_pretrained(slot_model_path).to(device).eval()
intent_model = AutoModelForSequenceClassification.from_pretrained(intent_model_path).to(device).eval()

def predict(sentence, tokenizer, slot_model, intent_model, label_list, intent_list):
    words = sentence.strip().split()
    tokenized = tokenizer(
        words,
        is_split_into_words=True,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128
    )

    for k in tokenized:
        tokenized[k] = tokenized[k].to(device)

    with torch.no_grad():
        # 인텐트 예측
        intent_logits = intent_model(**tokenized).logits
        intent_probs = F.softmax(intent_logits, dim=1)[0]
        intent_pred_id = torch.argmax(intent_probs).item()
        intent_label = intent_list[intent_pred_id]
        intent_score = intent_probs[intent_pred_id].item()

        # 슬롯 태깅 예측
        slot_logits = slot_model(**tokenized).logits
        slot_probs = F.softmax(slot_logits, dim=2)[0]
        slot_preds = torch.argmax(slot_probs, dim=1).tolist()
        slot_scores = slot_probs[range(len(slot_preds)), slot_preds].tolist()

    input_ids = tokenized["input_ids"][0].cpu()
    word_ids = tokenized.word_ids(batch_index=0)

    print(f"\n🟦 문장: {sentence}")
    print(f"🔸 예측 인텐트: {intent_label}  (score: {intent_score:.4f})")
    print(f"🔸 슬롯 태깅:")

    word_to_tag = {}
    for idx, word_id in enumerate(word_ids):
        if word_id is None or input_ids[idx].item() in tokenizer.all_special_ids:
            continue
        if word_id not in word_to_tag:
            pred_id = slot_preds[idx]
            score = slot_scores[idx]
            word_to_tag[word_id] = (label_list[pred_id], score)

    for i, word in enumerate(words):
        if i in word_to_tag:
            tag, score = word_to_tag[i]
            print(f"   {word:10} → {tag:20} (score: {score:.4f})")
        else:
            print(f"   {word:10} → [NO TAG]")

    print()

# ✅ 사용자 입력 반복
print("🟢 문장을 입력하세요. 'exit', 'quit', 'q' 입력 시 종료됩니다.\n")
while True:
    user_input = input("💬 입력 > ").strip()
    if user_input.lower() in ["exit", "quit", "q"]:
        print("👋 종료합니다.")
        break
    if user_input == "":
        continue
    predict(user_input, tokenizer, slot_model, intent_model, label_list, intent_list)


🟢 문장을 입력하세요. 'exit', 'quit', 'q' 입력 시 종료됩니다.



💬 입력 >  520번 버스를 타고 강남방면으로 가려하는데, 과천도서관에서 타려고 해. 언제쯤 도착할까?



🟦 문장: 520번 버스를 타고 강남방면으로 가려하는데, 과천도서관에서 타려고 해. 언제쯤 도착할까?
🔸 예측 인텐트: arrival_bus  (score: 1.0000)
🔸 슬롯 태깅:
   520번       → B-ROUTE              (score: 1.0000)
   버스를        → B-TRANSPORT-BUS      (score: 1.0000)
   타고         → O                    (score: 1.0000)
   강남방면으로     → B-DIRECTION          (score: 0.9995)
   가려하는데,     → O                    (score: 1.0000)
   과천도서관에서    → B-STATION            (score: 1.0000)
   타려고        → O                    (score: 1.0000)
   해.         → O                    (score: 1.0000)
   언제쯤        → O                    (score: 1.0000)
   도착할까?      → O                    (score: 1.0000)



💬 입력 >  센트로빌은 멋진 도시야. 



🟦 문장: 센트로빌은 멋진 도시야.
🔸 예측 인텐트: other  (score: 1.0000)
🔸 슬롯 태깅:
   센트로빌은      → B-STATION            (score: 1.0000)
   멋진         → O                    (score: 1.0000)
   도시야.       → O                    (score: 1.0000)



...


🟦 문장: 관문초등학교역에서 단국대쪽으로 가는 24번 버스를 타고싶은데 몇분에 와?
🔸 예측 인텐트: arrival_bus  (score: 1.0000)
🔸 슬롯 태깅:
   관문초등학교역에서  → B-STATION            (score: 1.0000)
   단국대쪽으로     → B-DIRECTION          (score: 0.9999)
   가는         → O                    (score: 1.0000)
   24번        → B-ROUTE              (score: 1.0000)
   버스를        → B-TRANSPORT-BUS      (score: 1.0000)
   타고싶은데      → O                    (score: 1.0000)
   몇분에        → O                    (score: 1.0000)
   와?         → O                    (score: 1.0000)



💬 입력 >  판교역에서 단국대정문역까지 가는 3030번 버스는 대체 언제올지 알려줄래?



🟦 문장: 판교역에서 단국대정문역까지 가는 3030번 버스는 대체 언제올지 알려줄래?
🔸 예측 인텐트: arrival_bus  (score: 1.0000)
🔸 슬롯 태깅:
   판교역에서      → B-STATION            (score: 1.0000)
   단국대정문역까지   → B-DIRECTION          (score: 0.9993)
   가는         → O                    (score: 1.0000)
   3030번      → B-ROUTE              (score: 1.0000)
   버스는        → B-TRANSPORT-BUS      (score: 1.0000)
   대체         → O                    (score: 1.0000)
   언제올지       → O                    (score: 1.0000)
   알려줄래?      → O                    (score: 1.0000)



💬 입력 >  q


👋 종료합니다.


In [ ]:
#과거 사용했던 각종 테스트 문장들
sentences = [
    "단국대입구 정류소에 720번 버스 지금 오고 있어?",
    "죽전역에서 신분당선 타면 강남 방면으로 얼마나 걸려?",
    "지금 9호선 급행열차 타면 여의도 쪽 사람 많아?",
    "오금역 근처에서 점심 뭐 먹지?",
    "110-2번 버스가 동인천 방향으로 곧 도착하나요?",
    "양재역에서 지하철 타고 삼성 방면 가는 거 얼마나 복잡할까?",
    "지하철역 플랫폼에 사람 너무 많아서 숨막힐 뻔했어.",
    "파란불 안 켜져 있으면 버스 안 오는 거야?",
    "강남역에서 타는 2호선 지금 헬게이트일까?",
    "7017번은 신설동역 근처 언제쯤 지나가나요?",
    "양재시민의숲역에 오늘 무슨 행사 있대요?",
    "3호선 구파발 방면 타면 사람 많을까요?",
    "강변역 2호선 지금 왔나?",
    "버스 771-A번 강남지나가?",
    "지금 타면 ㄱㅊ? 수유역 지하철 지났는지 아세요?",
    "9호선 급행 신논현ㅇ서 몇 정거장 남았냐",
    "이수역에서 내린 다음 3호선 탄댔는데 몇 번 출구가 가깝대?",
    "3호선 고속터미널역에서 잠실까지 가는 지하철은 얼마나 혼잡한가요?",
    "광화문역에 5호선 지금 들어오고 있는 중인가요?",
    "서울대입구역 정류소에 6511번 버스 곧 도착하나요?",
    "지금 시청역에서 탈 수 있는 2호선 열차는 어디까지 도착했나요?",
    "버스 7016번이 서울역 방면으로 지금 어느 위치인지 알 수 있을까요?",
    "양재 시민의 숲 역에서는 진짜 많은 버스들이 다니더라고",
    "양재 시민의 숲 정거장에서는 신분당선 지하철로 갈아탈 수 있지",
    "양재 근처 정거장들은 정말 사람들이 많아서 혼잡하더라.",
    "양재 시민의 숲에서 420번 버스를 타면 강남역 근처 맛집에 갈 수 있다더라.",
    "양재 시민의 숲에서 510 버스는 대체 언제 오는거야? 얼마나 기다려야 도착하는지 알아봐줄래?"]
